In [22]:
#!pip install robobrowser tabula-py

In [48]:
import json
import os
import requests
import tabula

# https://github.com/jmcarp/robobrowser/issues/93
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from robobrowser import RoboBrowser

from bs4 import BeautifulSoup

## MCA Director Disqualified List

In [49]:
PARENT_SOURCE = 'http://www.mca.gov.in'
SOURCE = 'http://www.mca.gov.in/MinistryV2/disqualifieddirectorslist.html'

In [50]:
br = RoboBrowser(history=True,
                 user_agent='Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
                 parser='lxml'
                 )

In [51]:
br.open(SOURCE)

In [55]:
html = br.response.content

In [56]:
soup = BeautifulSoup(html, 'lxml')

In [57]:
links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]

In [58]:
if not os.path.exists('files'):
    os.makedirs('files')

In [59]:
file_sources = {}

In [60]:
for link in links:
    br.open(PARENT_SOURCE + link)
    html = br.response.content
    soup = BeautifulSoup(html, 'lxml')
    file_links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]
    for file_link in file_links:
        FILE_URL = PARENT_SOURCE + file_link
        FILENAME = file_link.split('/')[-1]
        br.open(FILE_URL)
        if br.response.status_code != 200:
            break
        file_sources[FILENAME] = FILE_URL
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)

In [61]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [37]:
FILENAME = 'files/DisqualifiedDirectorsAhmedabadSearchable_02072020.pdf'

In [38]:
tables = tabula.read_pdf(FILENAME, pages='all', multiple_tables=True)

In [39]:
tables[0] # First page

,Sr No.,CIN,Company,DIN,Director,ROC Code,Status,Period\rfrom,Period Till
0,1,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,147642,JAYESHBHAI PATEL,RC202,Disqualified,01.11.16,31.10.21
1,2,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,53097,KULIN VYAS,RC202,Disqualified,01.11.16,31.10.21
2,3,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,6775586,ASHOKKUMAR PATEL,RC202,Disqualified,01.11.16,31.10.21
3,4,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276686,DHARMENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
4,5,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276710,NAVINCHANDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
5,6,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276729,BHUPENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
6,7,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276686,DHARMENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
7,8,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276710,NAVINCHANDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
8,9,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276729,BHUPENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
9,10,L15400GJ1992PLC017712,JASMINA INDUSTRIES LIMITED,3298444,OBULREDDY PUPPALA,RC202,Disqualified,01.11.16,31.10.21


Fields are `Sr No.`, `CIN`, `Company`, `DIN`, `Director`, `ROC code`, `Status`, `Period from`, `Period Till`
Globally we need `PAN`, `Name`, `AddedDate` (day of blacklisting according to the source),
`Source`, `Meta` (a JSON encoded field of whatever fields each source provides).